# Data import

Common tomography data formats are imported through [DXchange](http://dxchange.readthedocs.org/en/latest/source/api/dxchange.exchange.html) (available through Conda):


In [ ]:
import dxchange as dx

In [ ]:
# Load data from the file we created with the previous notebook
prj, flat, dark, ang = dx.read_aps_32id('data/data-simulated.h5')

# Tomographic image reconstruction

TomoPy implements a variety of algorithms which fall into two main categories: direct and iterative methods.

## Direct reconstruction algorithms

* Filtered-Backprojection with interpolation on Fourier space (Gridrec)
* Filtered-Backprojection with interpolation on real space (FBP)

In [ ]:
import tomopy
import matplotlib.pyplot as plt

# Make a dictionary to store all of the different reconstructions.
rec = dict()

In [ ]:
# Required parametes are the projection data and the relative angles of at which each projection was collected.
rec['gridrec'] = tomopy.recon(tomo=prj, theta=ang, algorithm='gridrec')

In [ ]:
plt.figure()
plt.imshow(rec['gridrec'][0])
plt.xlabel('x')
plt.ylabel('y')
plt.show()

Using optional parameters, can define the shape of the final image reconstruction:

In [ ]:
rec['gridrec'] = tomopy.recon(prj, ang, algorithm='gridrec', num_gridx=128, num_gridy=128)

In [ ]:
plt.figure()
plt.imshow(rec['gridrec'][0])
plt.xlabel('x')
plt.ylabel('y')
plt.show()

### Choosing filters

Gridrec can be filtered in order to smooth the reconstruction (filters were never implemented for FBP).

Available filters:

* Ramp
* Shepp-Logan
* Butterworth
* Parzen
* Cosine
* Hamming

In [ ]:
import numpy as np

# low-pass Butterworth filter parameters
cutoff = 0.5  # [0, 1]
order = 2  # 0, 1, 2, 3, ...

x = np.linspace(-1, 1, 128)
y = 1 / (1 + np.power(x / cutoff, 2 * order))

plt.figure()
plt.plot(x, y)
plt.axis('tight')
plt.grid(True)
plt.show()

In [ ]:
rec['gridrec'] = tomopy.recon(
    prj, ang, algorithm='gridrec', 
    num_gridx=128, num_gridy=128, 
    filter_name='butterworth', filter_par=[0.5, 2.],
)

In [ ]:
plt.figure()
plt.imshow(rec['gridrec'][0])
plt.xlabel('x')
plt.ylabel('y')
plt.show()

## Iterative reconstrucrtion algorithms

### Without penalization

#### Kaczmarz methods:

* Algebrais Reconstruction Technique (ART)
* Simultaneous Iterative Reconstruction Technique (SIRT)
* Block Algebraic Reconstruction (BART)

In [ ]:
for algorithm in ['art', 'sirt', 'bart']:
    rec[algorithm] = tomopy.recon(
            prj, ang, algorithm=algorithm, 
            num_gridx=128, num_gridy=128,
            num_iter=10, # these methods require iterations
    )

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.imshow(rec['art'][0])
plt.title('ART')
plt.xlabel('x')
plt.ylabel('y')
plt.subplot(132)
plt.imshow(rec['sirt'][0])
plt.title('SIRT')
plt.xlabel('x')
plt.ylabel('y')
plt.subplot(133)
plt.imshow(rec['bart'][0])
plt.title('BART')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

#### Expectation maximization algorithms:

* Maximum-Likelihood Expectation-Maximization (MLEM)
* Ordered Subset Expectation-Maximization (OSEM)

In [ ]:
for algorithm in ['mlem', 'osem']:
    rec[algorithm] = tomopy.recon(
        prj, ang, algorithm=algorithm, 
        num_gridx=128, num_gridy=128,
        num_iter=10
    )

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.imshow(rec['mlem'][0])
plt.title('MLEM')
plt.xlabel('x')
plt.ylabel('y')
plt.subplot(132)
plt.imshow(rec['osem'][0])
plt.title('OSEM')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

### With quadratic penalization

#### Expectation maximization algorithms:

* Penalized Maximum-Likelihood with Quadratic penalty (PML_QUAD)
* Ordered Subset Penalized Maximum-Likelihood with Quadratic penalty (OSMPL_QUAD)

In [ ]:
# PML_QUAD
rec['pml_quad'] = tomopy.recon(prj, ang, algorithm='pml_quad', 
                   num_gridx=128, num_gridy=128,
                   num_iter=20,
                   reg_par=0.1)

rec5 = tomopy.recon(prj, ang, algorithm='pml_quad', 
                   num_gridx=128, num_gridy=128,
                   num_iter=20,
                   reg_par=100)

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.imshow(rec['pml_quad'][0])
plt.title('PML_QUAD, reg_par=0.1')
plt.xlabel('x')
plt.ylabel('y')
plt.subplot(132)
plt.imshow(rec5[0])
plt.title('PML_QUAD, reg_par=100')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

### With hybrid (mixed quadratic-linear) penalization

#### Expectation maximization algorithms:

* Penalized Maximum-Likelihood with Hybrid penalty (PML_HYBRID)
* Ordered Subset Penalized Maximum-Likelihood with Hybrid penalty (OSMPL_QUAD)

In [ ]:
# OSPML_HYBRID
rec6 = tomopy.recon(prj, ang, algorithm='ospml_hybrid', 
                   num_gridx=128, num_gridy=128,
                   num_iter=20,
                   reg_par=[1, 0.1])

rec7 = tomopy.recon(prj, ang, algorithm='ospml_hybrid', 
                   num_gridx=128, num_gridy=128,
                   num_iter=20,
                   reg_par=[0.1, 1],
                   num_block=10)

rec['ospml_hybrid'] = tomopy.recon(prj, ang, algorithm='ospml_hybrid', 
                   num_gridx=128, num_gridy=128,
                   num_iter=20,
                   reg_par=[0.1, 0.1],
                   num_block=10)

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.imshow(rec6[0])
plt.title('OSPML_HYBRID, reg_par=[1, 0.1]')
plt.xlabel('x')
plt.ylabel('y')
plt.subplot(132)
plt.imshow(rec7[0])
plt.title('OSPML_HYBRID, reg_par=[0.1, 1]')
plt.xlabel('x')
plt.ylabel('y')
plt.subplot(133)
plt.imshow(rec['ospml_hybrid'][0])
plt.title('OSPML_HYBRID, reg_par=[0.1, 0.1]')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

# ACTIVITY: Initialization of iterative algorithms

In the cell below, compare an iterative algorithm initalized with a constant value with one initialized with a direct inversion algorithm. 

In [ ]:
direct = [[[0]]]
initialized_constant = [[[0]]]
initialized_direct = [[[0]]]

In [ ]:
plt.figure(figsize=(12, 4))
plt.subplot(131)
plt.imshow(direct[0])
plt.title('Direct method')
plt.xlabel('x')
plt.ylabel('y')
plt.subplot(132)
plt.imshow(initialized_constant[0])
plt.title('Initiated with constant vals.')
plt.xlabel('x')
plt.subplot(133)
plt.ylabel('y')
plt.imshow(initialized_direct[0])
plt.title('Initiated with direct method')
plt.xlabel('x')
plt.ylabel('y')
plt.show()